In [1]:
# Python SQL toolkit and Object Relational Mapper
import pandas as pd
from datetime import date

import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.orm import Session

from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Text, Integer, String, Date, Float

# PyMySQL 
import pymysql
pymysql.install_as_MySQLdb()

In [2]:
# Path of the CSV file
measurements = "Resources/clean_hawaii_measurements.csv"
stations="Resources/clean_hawaii_stations.csv"

# Read CSV file into a pandas DataFrame
measurements_df = pd.read_csv(measurements, dtype=object)
stations_df=pd.read_csv(stations, dtype=object)
stations_df.head()

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3
1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7
3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


In [3]:
# Create an engine to a SQLite database file called `hawaii.sqlite`
engine = create_engine("sqlite:///hawaii.sqlite")

In [4]:
# Create a connection to the engine called `conn`
conn = engine.connect()

In [5]:
# Use `declarative_base` from SQLAlchemy to model the Measurements table and Stations table as an ORM class
Base = declarative_base()

class Measurement(Base):
    __tablename__ = 'measurement'

    id = Column(Integer, primary_key=True)
    station = Column(String(225))
    date = Column(Text)
    prcp=Column(Float)
    tobs=Column(Integer)
    def __repr__(self):
        return f"id={self.id}, station={self.station}"
    
class Station(Base):
    __tablename__ = 'station'

    id = Column(Integer, primary_key=True)
    station = Column(String(255))
    name = Column(Text)
    latitude=Column(Float)
    longitude=Column(Float)  
    elevation=Column(Float)
    def __repr__(self):
        return f"id={self.id}, name={self.name}"

In [6]:
# Use `create_all` to create the measurements and stations tables in the database
Base.metadata.create_all(engine)

In [8]:
# Use Orient='records' to create a list of data to write
# to_dict() cleans out DataFrame metadata as well
# http://pandas-docs.github.io/pandas-docs-travis/io.html#orient-options
measurement_data = measurements_df.to_dict(orient='records')
measurement_data[0]

{'date': '2010-01-01', 'prcp': '0.08', 'station': 'USC00519397', 'tobs': '65'}

In [9]:
# Use Orient='records' to create a list of data to write
# to_dict() cleans out DataFrame metadata as well
# http://pandas-docs.github.io/pandas-docs-travis/io.html#orient-options
station_data = stations_df .to_dict(orient='records')
station_data[0]

{'elevation': '3',
 'latitude': '21.2716',
 'longitude': '-157.8168',
 'name': 'WAIKIKI 717.2, HI US',
 'station': 'USC00519397'}

In [10]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [13]:
# Save the reference to the `Measurements` table as a variable called `measure_tbl`
measurement_tbl = sqlalchemy.Table('Measurement', metadata, autoload=True)

In [14]:
# Save the reference to the `Station` table as a variable called `station_tbl`
station_tbl = sqlalchemy.Table('Station', metadata, autoload=True)

In [16]:
# Use `table.delete()` to remove any pre-existing data.
# Note that this is a convenience function so that you can re-run the example code multiple times.
# You would not likely do this step in production.
conn.execute(measurement_tbl.delete())
conn.execute(station_tbl.delete())

In [17]:
# Use `measurement_tbl.insert()` and `station_tbl.insert()` to insert the data into the tables
conn.execute(measurement_tbl.insert(), measurement_data)
conn.execute(station_tbl.insert(), station_data)

In [19]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from Measurement limit 5").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65),
 (2, 'USC00519397', '2010-01-02', 0.0, 63),
 (3, 'USC00519397', '2010-01-03', 0.0, 74),
 (4, 'USC00519397', '2010-01-04', 0.0, 76),
 (5, 'USC00519397', '2010-01-06', 0.0, 73)]

In [18]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from Station limit 5").fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6)]